# General remarks from preprocessing stage

It has been studied previously, that the membership data, the user log data, and the training member data (including the churn flag) contain no duplicates. Duplicates can be found in the transaction data, those have been removed before aggregating information. The user log data is too large to be dealt with in one go using pandas, due to insufficient memory of my laptop, thus aggregation has to be dealt with in chunks in pandas. All of that has been performed in the DataProcessor_KKBox notebook.

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

from sklearn import metrics
import datetime as dt
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

import seaborn as sns

import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.io as pio

## User Activity Time Evolution

Check activity of users by user logs, since we care about churn as inactivity of 30 days, group the user logs in intervals of 30 days. Churn is in fact defined as no renewal of subscription 30 days after a previous membership has been expired. In fact a membership might be ongoing, but a user shows now activity in terms of songs listened to. But user acitivity in terms of played songs can give us an idea about churn, retained or reactivated active users.

In [2]:
df_ul=pd.read_csv("data/user_logs.csv",usecols=['msno','date'])
df_ul["date"]=pd.to_datetime(df_ul["date"], format='%Y%m%d', errors='ignore')
df_ul.head()

,msno,date
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-05-13
1,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-07-09
2,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-01-05
3,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-03-06
4,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-05-01


In [3]:
df_ul_v2=pd.read_csv("data/user_logs_V2.csv",usecols=['msno','date'])
df_ul_v2["date"]=pd.to_datetime(df_ul_v2["date"], format='%Y%m%d', errors='ignore')
df_ul_v2

,msno,date
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,2017-03-31
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,2017-03-30
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,2017-03-31
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,2017-03-31
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,2017-03-31
...,...,...
18396357,FGpiy2mB+vXLKziYRcY/xJcJEFJfRDfUqlU+p760f7E=,2017-03-14
18396358,iZRjKNMrw5ffEbfXODLhV/0tJLPbOH3am1WYDgqBf8Q=,2017-03-06
18396359,yztw4Y0EggG0w2wPkbMZx7ke7saSx7dLSfMheHZG/DQ=,2017-03-31
18396360,swCHwkNx30/aENjq30qqaLlm7bUUytbMXdz1bH7g0Jk=,2017-03-07


In [4]:
df_ul=df_ul.append(df_ul_v2)
df_ul

,msno,date
0,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-05-13
1,rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=,2015-07-09
2,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-01-05
3,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-03-06
4,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,2015-05-01
...,...,...
18396357,FGpiy2mB+vXLKziYRcY/xJcJEFJfRDfUqlU+p760f7E=,2017-03-14
18396358,iZRjKNMrw5ffEbfXODLhV/0tJLPbOH3am1WYDgqBf8Q=,2017-03-06
18396359,yztw4Y0EggG0w2wPkbMZx7ke7saSx7dLSfMheHZG/DQ=,2017-03-31
18396360,swCHwkNx30/aENjq30qqaLlm7bUUytbMXdz1bH7g0Jk=,2017-03-07


In [5]:
min_date=df_ul['date'].min()

In [6]:
#df_ul.groupby(["msno",pd.Grouper(key="date",freq="30D", origin='2015-01-01')])

In [7]:
max_date=df_ul['date'].max()

Using the full logging information in terms of active days seems not to work as desired due to high memory consumption while grouping the dates in 30 day intervals, checking on how many days the users had been active. Try now to operate in junks of 180 days, and grouping those per user to reduce the overall load, aka always 6 group periods and then sum up all at the end.

In [8]:
day_period=90
n_time_periods=math.ceil((max_date-min_date)/dt.timedelta(days=day_period))
time_periods=[min_date + i*dt.timedelta(days=day_period) for i in range (0,n_time_periods+1)]
time_periods

[Timestamp('2015-01-01 00:00:00'),
 Timestamp('2015-04-01 00:00:00'),
 Timestamp('2015-06-30 00:00:00'),
 Timestamp('2015-09-28 00:00:00'),
 Timestamp('2015-12-27 00:00:00'),
 Timestamp('2016-03-26 00:00:00'),
 Timestamp('2016-06-24 00:00:00'),
 Timestamp('2016-09-22 00:00:00'),
 Timestamp('2016-12-21 00:00:00'),
 Timestamp('2017-03-21 00:00:00'),
 Timestamp('2017-06-19 00:00:00')]

In [9]:
df_activity_per_user_list=[]
for tp in range(0,len(time_periods)-1):
    print("address periods",time_periods[tp],time_periods[tp+1])
    df_ul_in_period=df_ul[(df_ul['date']>=time_periods[tp]) & (df_ul['date']<time_periods[tp+1])].groupby(["msno",pd.Grouper(
        key="date",freq="30D", origin='2015-01-01')]).agg(
        activedays_per_msno=('msno','count')).reset_index()
    df_activity_per_user_list.append(df_ul_in_period)

address periods 2015-01-01 00:00:00 2015-04-01 00:00:00
address periods 2015-04-01 00:00:00 2015-06-30 00:00:00
address periods 2015-06-30 00:00:00 2015-09-28 00:00:00
address periods 2015-09-28 00:00:00 2015-12-27 00:00:00
address periods 2015-12-27 00:00:00 2016-03-26 00:00:00
address periods 2016-03-26 00:00:00 2016-06-24 00:00:00
address periods 2016-06-24 00:00:00 2016-09-22 00:00:00
address periods 2016-09-22 00:00:00 2016-12-21 00:00:00
address periods 2016-12-21 00:00:00 2017-03-21 00:00:00
address periods 2017-03-21 00:00:00 2017-06-19 00:00:00


In [11]:
large_df_activity_30day_intervalslarge_df_activity_30day_intervalslarge_df_activity_30day_intervals = pd.concat(df_activity_per_user_list, ignore_index=True)
large_df_activity_30day_intervals

,msno,date,activedays_per_msno
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2015-01-01,20
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2015-01-31,15
2,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2015-03-02,30
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,2015-01-01,12
4,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,2015-01-31,9
...,...,...,...
28702517,zzy0oyiTnRTo5Mbg23oKbBkf9eoaS7+eU4V+d14bzfY=,2017-03-21,8
28702518,zzy7iqSpfcRq7R4hmKKuhI+CJRs79a6pteqEggpiNO0=,2017-03-21,11
28702519,zzyHq6TK2+cBkeGFUHvh12Z7UxFZiSM7dOOSllSBPDw=,2017-03-21,10
28702520,zzz1Dc3P9s53HAowRTrm3fNsWju5yeN4YBfNDq7Z99Q=,2017-03-21,5


In [12]:
df_user_act_retention_30days = pd.crosstab(large_df_activity_30day_intervals['msno'], large_df_activity_30day_intervals['date'].dt.date).reset_index()

df_30days_mindate_of_msno = large_df_activity_30day_intervals.groupby('msno')['date'].min().reset_index()
df_30days_mindate_of_msno=df_30days_mindate_of_msno.rename(columns={"date": "min_activity_date"})

df_user_act_retention_30days= pd.merge(df_user_act_retention_30days,df_30days_mindate_of_msno,on="msno")
df_user_act_retention_30days

,msno,2015-01-01,2015-01-31,2015-03-02,2015-04-01,2015-05-01,2015-05-31,2015-06-30,2015-07-30,2015-08-29,...,2016-07-24,2016-08-23,2016-09-22,2016-10-22,2016-11-21,2016-12-21,2017-01-20,2017-02-19,2017-03-21,min_activity_date
0,+++4vcS9aMH7KWdfh5git6nA5fC5jjisd5H/NcM++WM=,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2015-04-01
1,+++EI4HgyhgcJHIPXk/VRP7bt17+2joG39T6oEfJ+tc=,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-03-26
2,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2016-08-23
3,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,2015-01-01
4,+++TipL0Kt3JvgNE9ahuJ8o+drJAnQINtxD4c5GePXI=,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2015-12-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5339417,zzzqx+aMPSFYjW71JqJ6T/hita+iVemVWzJTE4yQRx8=,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2016-02-25
5339418,zzztPAN9xjMytpZ0RN2gU9mScDULJnHQZK8eZb4uELU=,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2015-01-01
5339419,zzztsqkufVj9DPVJDM3FxDkhlbCL5z4aiYxgPSGkIK4=,0,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2015-01-31
5339420,zzzueVTwIa5TjXnG2c77bohCVkuksqLkd5mQTP0wTwQ=,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2017-02-19


In [13]:
check_dates_30d = df_user_act_retention_30days.columns[1:-1]
print(check_dates_30d)

Index([2015-01-01, 2015-01-31, 2015-03-02, 2015-04-01, 2015-05-01, 2015-05-31,
       2015-06-30, 2015-07-30, 2015-08-29, 2015-09-28, 2015-10-28, 2015-11-27,
       2015-12-27, 2016-01-26, 2016-02-25, 2016-03-26, 2016-04-25, 2016-05-25,
       2016-06-24, 2016-07-24, 2016-08-23, 2016-09-22, 2016-10-22, 2016-11-21,
       2016-12-21, 2017-01-20, 2017-02-19, 2017-03-21],
      dtype='object')


In [15]:
retention_array = []
for i in range(len(check_dates_30d)-1):
    retention_data = {}
    selected_check_date = check_dates_30d[i+1]
    prev_check_date = check_dates_30d[i]
    retention_data['min_activity_date'] = selected_check_date
    retention_data['TotalActiveUserCount'] = df_user_act_retention_30days[selected_check_date].sum()
    retention_data['NewUserCount'] = df_user_act_retention_30days[(df_user_act_retention_30days[selected_check_date]>0) & (selected_check_date==df_user_act_retention_30days['min_activity_date'].dt.date)][selected_check_date].sum()
    retention_data['RetainedUserCount'] = df_user_act_retention_30days[(df_user_act_retention_30days[selected_check_date]>0) & (df_user_act_retention_30days[prev_check_date]>0)][selected_check_date].sum()
    retention_data['InactiveUserCount'] = df_user_act_retention_30days[(df_user_act_retention_30days[selected_check_date]==0) & (df_user_act_retention_30days[prev_check_date]>0)][selected_check_date].count()
    #Customers reactivated after at least 1 period of inactivity
    retention_data['Reactivated_1Period_PLUS_UserCount'] = df_user_act_retention_30days[(df_user_act_retention_30days[selected_check_date]>0) & (df_user_act_retention_30days[prev_check_date]==0) & (prev_check_date>df_user_act_retention_30days['min_activity_date'].dt.date)][selected_check_date].count()
    # Customers reactivated after at least 2 periods of inactivity, we can only check that if we are two periods in
    if (i>0):
        second_prev_check_date = check_dates_30d[i-1] # if months[i-1].isna() == False else 0
        retention_data['Reactivated_2Period_PLUS_UserCount'] = df_user_act_retention_30days[(df_user_act_retention_30days[selected_check_date]>0) & (df_user_act_retention_30days[prev_check_date]==0) & (df_user_act_retention_30days[second_prev_check_date]==0) & (prev_check_date>df_user_act_retention_30days['min_activity_date'].dt.date)][selected_check_date].count()
    else:
        retention_data['Reactivated_2Period_PLUS_UserCount'] = 0
    # Customers reactivated after at least 3 periods of inactivity
    if (i>1):
        second_prev_check_date = check_dates_30d[i-1] # if months[i-1].isna() == False else 0
        third_prev_check_date = check_dates_30d[i-2] # if months[i-2].isna() == False else 0
        retention_data['Reactivated_3Period_PLUS_UserCount'] = df_user_act_retention_30days[(df_user_act_retention_30days[selected_check_date]>0) & (df_user_act_retention_30days[prev_check_date]==0) & (df_user_act_retention_30days[second_prev_check_date]==0) & (df_user_act_retention_30days[third_prev_check_date]==0) & (prev_check_date>df_user_act_retention_30days['min_activity_date'].dt.date)][selected_check_date].count()
    else:
        retention_data['Reactivated_3Period_PLUS_UserCount'] = 0
        
    # Customers reactivated after at ONLY 1 period of inactivity
    if(i==0):
        retention_data['Reactivated_1Period_ONLY_UserCount'] = df_user_act_retention_30days[(df_user_act_retention_30days[selected_check_date]>0) & (df_user_act_retention_30days[prev_check_date]==0) &  (prev_check_date>df_user_act_retention_30days['min_activity_date'].dt.date)][selected_check_date].count()
    else: 
        retention_data['Reactivated_1Period_ONLY_UserCount'] = df_user_act_retention_30days[(df_user_act_retention_30days[selected_check_date]>0) & (df_user_act_retention_30days[prev_check_date]==0) & (df_user_act_retention_30days[second_prev_check_date]>0) & (prev_check_date>df_user_act_retention_30days['min_activity_date'].dt.date)][selected_check_date].count() 
    
    # Customers reactivated after at ONLY 2 periods of inactivity
    if(i>0):
        second_prev_check_date = check_dates_30d[i-1] # if months[i-1].isna() == False else 0
        retention_data['Reactivated_2Period_ONLY_UserCount'] = df_user_act_retention_30days[(df_user_act_retention_30days[selected_check_date]>0) & (df_user_act_retention_30days[prev_check_date]==0) & (df_user_act_retention_30days[second_prev_check_date]==0) & (df_user_act_retention_30days[third_prev_check_date]>0) & (prev_check_date>df_user_act_retention_30days['min_activity_date'].dt.date)][selected_check_date].count()
    else:
        retention_data['Reactivated_2Period_ONLY_UserCount'] = 0
    # Customers reactivated after at ONLY 3 periods of inactivity
    third_prev_check_date = check_dates_30d[i-2] # if months[i-2].isna() == False else 0
    if(i>2):
        second_prev_check_date = check_dates_30d[i-1] # if months[i-1].isna() == False else 0
        fourth_prev_check_date = check_dates_30d[i-3] # if months[i-2].isna() == False else 0
        third_prev_check_date = check_dates_30d[i-2] # if months[i-2].isna() == False else 0
        retention_data['Reactivated_3Period_ONLY_UserCount'] = df_user_act_retention_30days[(df_user_act_retention_30days[selected_check_date]>0) & (df_user_act_retention_30days[prev_check_date]==0) & (df_user_act_retention_30days[second_prev_check_date]==0) & (df_user_act_retention_30days[third_prev_check_date]==0) & (prev_check_date>df_user_act_retention_30days['min_activity_date'].dt.date)][selected_check_date].count()
    else:
        retention_data['Reactivated_3Period_ONLY_UserCount'] = 0
    retention_array.append(retention_data)

In [16]:
tx_retention_plot = pd.DataFrame(retention_array)
tx_retention_plot["TotalActiveUserCount_prevPeriod"]=tx_retention_plot["TotalActiveUserCount"].shift(1)
tx_retention_plot["ActiveUserRate"]=tx_retention_plot["TotalActiveUserCount"]/tx_retention_plot["TotalActiveUserCount_prevPeriod"]
tx_retention_plot["NewUserRate"]=tx_retention_plot["NewUserCount"]/tx_retention_plot["TotalActiveUserCount_prevPeriod"]
tx_retention_plot["RetainedUserRate"]=tx_retention_plot["RetainedUserCount"]/tx_retention_plot["TotalActiveUserCount_prevPeriod"]
tx_retention_plot["ChurnRate"]=tx_retention_plot["InactiveUserCount"]/tx_retention_plot["TotalActiveUserCount_prevPeriod"]
tx_retention_plot["ReactivationRate"]=tx_retention_plot["Reactivated_1Period_PLUS_UserCount"]/tx_retention_plot["TotalActiveUserCount_prevPeriod"]
#print(np.max(tx_retention_plot["TotalActiveUserCount"]-tx_retention_plot["NewUserCount"]-tx_retention_plot["RetainedUserCount"]-tx_retention_plot["Reactivated_1Period_PLUS_UserCount"]))
tx_retention_plot.head(50)

,min_activity_date,TotalActiveUserCount,NewUserCount,RetainedUserCount,InactiveUserCount,Reactivated_1Period_PLUS_UserCount,Reactivated_2Period_PLUS_UserCount,Reactivated_3Period_PLUS_UserCount,Reactivated_1Period_ONLY_UserCount,Reactivated_2Period_ONLY_UserCount,Reactivated_3Period_ONLY_UserCount,TotalActiveUserCount_prevPeriod,ActiveUserRate,NewUserRate,RetainedUserRate,ChurnRate,ReactivationRate
0,2015-01-31,948986,210603,738383,191354,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-03-02,936596,172092,739768,209218,24736,0,0,24736,0,0,948986.0,0.986944,0.181343,0.779535,0.220465,0.026066
2,2015-04-01,939930,145035,751170,185426,43725,13984,0,29741,13984,0,936596.0,1.003560,0.154853,0.802021,0.197979,0.046685
3,2015-05-01,917844,118848,754980,184950,44016,19584,6842,24432,12742,6842,939930.0,0.976503,0.126443,0.803230,0.196770,0.046829
4,2015-05-31,918345,116150,751769,166075,50426,23933,12765,26493,11168,12765,917844.0,1.000546,0.126547,0.819060,0.180940,0.054940
5,2015-06-30,866356,119409,713611,204734,33336,15762,8462,17574,7300,8462,918345.0,0.943388,0.130026,0.777062,0.222938,0.036300
6,2015-07-30,907676,167353,703728,162628,36595,17419,10249,19176,7170,10249,866356.0,1.047694,0.193169,0.812285,0.187715,0.042240
7,2015-08-29,907009,137981,730609,177067,38419,20697,11783,17722,8914,11783,907676.0,0.999265,0.152016,0.804923,0.195077,0.042327
8,2015-09-28,992513,214981,742114,164895,35418,20591,13955,14827,6636,13955,907009.0,1.094270,0.237022,0.818199,0.181801,0.039049
9,2015-10-28,1039647,242735,761638,230875,35274,20192,14093,15082,6099,14093,992513.0,1.047490,0.244566,0.767383,0.232617,0.035540


In [24]:
from datetime import datetime, timedelta,date

tx_retention_plot=tx_retention_plot[(tx_retention_plot["min_activity_date"]>datetime.strptime("2015-01-01", '%Y-%m-%d').date())
                                & (tx_retention_plot["min_activity_date"]<datetime.strptime("2017-03-21", '%Y-%m-%d').date())]

In [26]:
plot_data = [
    go.Scatter(
        x=tx_retention_plot['min_activity_date'],
        y=tx_retention_plot['TotalActiveUserCount'],
        name="TotalActiveUserCount"
    ),
    go.Scatter(
        x=tx_retention_plot['min_activity_date'],
        y=tx_retention_plot['NewUserCount'],
        name="NewUserCount"
    ),
    go.Scatter(
        x=tx_retention_plot['min_activity_date'],
        y=tx_retention_plot['RetainedUserCount'],
        name="RetainedUserCount"
    ),
    # Inactive customers keep adding up as we know from Recency, therefore this is not shown to avoid squeezing the chart
    go.Scatter(
       x=tx_retention_plot['min_activity_date'],
       y=tx_retention_plot['InactiveUserCount'],
       name="ChurnedUserCount"
    ),
    go.Scatter(
        x=tx_retention_plot['min_activity_date'],
        y=tx_retention_plot['Reactivated_1Period_PLUS_UserCount'],
        name="ReactivatedUserCount"
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Periodical (30 days) Users Evolution'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)
pio.write_image(fig, 'figures/KKBox_30d_UserEvolution_AllAbsoluteNumbers.png')

In [30]:
tx_retention_plot=tx_retention_plot[(tx_retention_plot["min_activity_date"]>datetime.strptime("2015-01-31", '%Y-%m-%d').date())
                                & (tx_retention_plot["min_activity_date"]<datetime.strptime("2017-03-21", '%Y-%m-%d').date())]

In [33]:
plot_data = [

    go.Scatter(
        x=tx_retention_plot['min_activity_date'],
        y=tx_retention_plot['RetainedUserRate'],
        name="Retained User Rate"
    ),
    go.Scatter(
        x=tx_retention_plot['min_activity_date'],
        y=tx_retention_plot['ReactivationRate'],
        name="Reactivated User Rate"
    ),
    go.Scatter(
        x=tx_retention_plot['min_activity_date'],
        y=tx_retention_plot['ChurnRate'],
        name="Churn Rate"
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Periodical (30 days) Users Evolution',
        yaxis={"tickformat": ',.0%',
                "range": [0,1.0]}
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.update_layout(
    font=dict(size=16),
    legend=dict(
    #title_text='Rate To Previous Active Users',
    yanchor="top",
    y=0.75,
    xanchor="right",
    x=0.80
))
pyoff.iplot(fig)
pio.write_image(fig, 'figures/KKBox_30d_UserEvolution_30d_UserEvolution_Retained_Reactivation_Rates.png')

### Overall User Evolution

Studied in periods of 30 days the retained user rate is rather flat around 80 %, the reactivated user rate is typically of the order of 3-4 %, thus the churn rate is around 20 %, slightly lower in the first quarter of 2017. Definitions are defined as activity in terms of users streaming songs.

In [34]:
df_mem=pd.read_csv("data/members_v3.csv")
df_mem

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915
...,...,...,...,...,...,...
6769468,VSGkb3hyBRUtb/b1MQUZbvOkktS3vKLnhMHW0CF8eyU=,1,0,NaN,7,20151020
6769469,nWjH7glPkZ7jOVaCRwwjlpmp0T1hSWdv8hMJxiWCwKc=,1,0,NaN,7,20151020
6769470,GH+b5+1tlv7ZZXsA8upBzVXMTLyffKcsF7WoU8b5rOI=,15,26,female,4,20151020
6769471,XVlwT3fdCFGKqerEKBzUIjK+jzI6jzSke4cDMVhYyjE=,1,0,NaN,4,20151020


In [51]:
df_mem["registration_init_time_as_dt"]=pd.to_datetime(df_mem["registration_init_time"], format='%Y%m%d', errors='ignore')
df_mem.sort_values(by="registration_init_time")

,msno,city,bd,gender,registered_via,registration_init_time,registration_init_time_as_dt
2778759,+1VIk9gpQ9cOLXPbQOxeLm2VjH9oKWtHyIMwe+BfFt0=,21,32,male,9,20040326,2004-03-26
6363176,yCQADf9WWjAUSBRJHtZH31WrmL6MsxpBo165UMpUA6I=,10,0,female,9,20040326,2004-03-26
82956,XT0yF1DRjbn5ymbnsasJnag4+53huda0TZ3bRPIcrAA=,13,32,male,9,20040326,2004-03-26
107040,OVR9+ssQRSxtzfQjcVtdoiZ6c4bHIuLdhUTVdA9oR0M=,13,30,female,9,20040326,2004-03-26
107041,3kM1IV7sTY3OIPNt/8IG1Ahwlg57T7/BOL1VzVUB4HM=,4,39,male,9,20040326,2004-03-26
...,...,...,...,...,...,...,...
5152046,Np0Xkl+nRyBKgIZh1z77he6Fis27DNQTk8F5IJc4Rn8=,1,40,female,3,20170429,2017-04-29
5152045,xl587vDc3ZhkRvA+hk9PI7B8MOZxDBrikbOsou5w3Nc=,1,0,NaN,7,20170429,2017-04-29
5152044,FN00EIM5kDR3aR8rRoRVWlBIHoNZSdlHSgOSSzfL5ng=,1,46,male,3,20170429,2017-04-29
252864,8hYHwm6tMWzkqBd+26xr1QfcuHrSpq63Cj2IaB7XGFc=,1,21,male,3,20170429,2017-04-29


In [35]:
print("age 0",len(df_mem[df_mem["bd"]==0]),"neg age",len(df_mem[df_mem["bd"]<0]), "age 0<10",len(df_mem[(df_mem["bd"]>0) & (df_mem["bd"]<10)]),
     "age 10-20", len(df_mem[(df_mem["bd"]>=10) & (df_mem["bd"]<20)]),"age 20-50",len(df_mem[(df_mem["bd"]>=20) & (df_mem["bd"]<50)]),
     "age 50-100",len(df_mem[(df_mem["bd"]>=50) & (df_mem["bd"]<100)]),"age >100",len(df_mem[df_mem["bd"]>100]))

age 0 4540215 neg age 274 age 0<10 899 age 10-20 330391 age 20-50 1779048 age 50-100 113126 age >100 5377


In [55]:
1937764/2426143

0.7987014780249969

Age (bd column) offers no useful information as well, for about two thirds of customers the age is 0, also impossible ages are listed, thus will ignore that information for now. For 5 339 422 out of the 6 769 473 users we have the user log information available. Only for 2 425 143 users we have transaction data available.

After merging the user log data and the membership data we have 5 169 234 members left (96.8 % of the member for which we have user logs), after merging of the transaction data we are left with 79.9 % of the original transaction data. In user log overall data we have a couple of users which have 1 active day, for those standard deviations of distributions is not defined properly.

It has been checked that both transaction data and user log data contain information starting from Jan 1, 2015 to March 31, 2017.

In [53]:
df_user_log_agg_read=pd.read_csv("df_user_log_aggregated_per_user_def.csv")
df_user_log_agg_read=df_user_log_agg_read.drop(columns="Unnamed: 0")
df_user_log_agg_read["last_date_active"]=pd.to_datetime(df_user_log_agg_read["last_date_active"], format='%Y-%m-%d', errors='ignore')
print(df_user_log_agg_read.dtypes)
df_user_log_agg_read

msno                           object
activity_duration               int64
last_date_active       datetime64[ns]
n_active_days                   int64
sum_num_25                      int64
sum_num_50                      int64
sum_num_75                      int64
sum_num_985                     int64
sum_num_100                     int64
sum_num_unq                     int64
tot_secs                      float64
mean_num_25                   float64
median_num_25                 float64
q25_num_25                    float64
q75_num_25                    float64
std_num_25                    float64
median_num_50                 float64
std_num_50                    float64
median_num_75                 float64
std_num_75                    float64
median_num_985                float64
std_num_985                   float64
mean_num_100                  float64
median_num_100                float64
q25_num_100                   float64
q75_num_100                   float64
std_num_100 

,msno,activity_duration,last_date_active,n_active_days,sum_num_25,sum_num_50,sum_num_75,sum_num_985,sum_num_100,sum_num_unq,...,n_max_break,n_min_break,n_mean_break,n_mode_break,n_median_break,n_max_break_last,n_min_break_last,n_mean_break_last,n_mode_break_last,n_median_break_last
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,6,2016-09-15,7,60,12,14,7,171,179,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,+++dz9ZCWE2HB/47pJU82NJXQzQuZDx1Wm50YSk/kKk=,3,2016-03-02,4,5,1,1,1,183,147,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,++5IYGT7+CWMJ8hRsqoQLoaTXBLMJzlfK12eMTr9Ilw=,118,2015-05-01,63,9,5,0,3,349,191,...,5.0,0.0,0.903226,0.0,0.0,5.0,0.0,2.444444,0.0,3.0
3,++7jYuHyUSp41PyuttFx/MCepv7TdFQULgN8TxZULZk=,542,2016-06-30,183,669,115,82,79,1790,2250,...,25.0,0.0,1.978022,0.0,1.0,16.0,14.0,15.000000,14.0,15.0
4,++92FghbCQPqDmQ96QzNiuEMoDxrMOmuaisu1UCrYn0=,347,2017-01-22,279,465,138,126,143,10690,10699,...,6.0,0.0,0.248201,0.0,0.0,6.0,0.0,0.875000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5339417,zzjrmRFJWpHm5f6OcUTHhnj3Xp6rQhaVC4isJBbjm+8=,14,2017-03-24,8,22,6,2,1,135,157,...,4.0,0.0,1.000000,0.0,1.0,4.0,0.0,1.000000,0.0,1.0
5339418,zzpujC8UYr82W9wURtuMOI+hCaskgYEluKz1TJKFoEY=,2,2017-03-16,3,4,0,0,0,149,112,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
5339419,zzu6u0H6kxDzwhPY7KpdvV4CbMsjIbBVSM6fYXwTrE4=,0,2017-03-09,1,7,2,0,1,11,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5339420,zzxIjlrMxRzNVDD6XqUDRBqkWn7OJ9mzFmFQlfLtLCE=,26,2017-03-31,12,11,10,3,3,27,38,...,7.0,0.0,1.363636,0.0,0.0,7.0,0.0,1.363636,0.0,0.0


In [43]:
"""
df_tra=pd.read_csv("data/transactions.csv")
df_tra_V2=pd.read_csv("data/transactions_v2.csv")
df_tra=df_tra.append(df_tra_V2)
df_tra=df_tra.drop_duplicates(ignore_index=True)
df_tra['msno'].nunique()
"""

2426143

In [48]:
df_tra_agg_per_user=pd.read_csv("df_transaction_aggregated_per_user_def_with_duration_NEW.csv")
df_tra_agg_per_user=df_tra_agg_per_user.drop(columns="Unnamed: 0")
df_tra_agg_per_user["max_date_trans"]=pd.to_datetime(df_tra_agg_per_user["max_date_trans"], format='%Y-%m-%d', errors='ignore')
df_tra_agg_per_user["min_date_trans"]=pd.to_datetime(df_tra_agg_per_user["min_date_trans"], format='%Y-%m-%d', errors='ignore')
df_tra_agg_per_user["max_memb_exp"]=pd.to_datetime(df_tra_agg_per_user["max_memb_exp"], format='%Y-%m-%d', errors='ignore')
df_tra_agg_per_user["first_cancel_membership_expire_date"]=pd.to_datetime(df_tra_agg_per_user["first_cancel_membership_expire_date"], format='%Y-%m-%d', errors='ignore')
df_tra_agg_per_user["last_cancel_membership_expire_date"]=pd.to_datetime(df_tra_agg_per_user["last_cancel_membership_expire_date"], format='%Y-%m-%d', errors='ignore')
df_tra_agg_per_user["first_auto_renew_transaction"]=pd.to_datetime(df_tra_agg_per_user["first_auto_renew_transaction"], format='%Y-%m-%d', errors='ignore')
df_tra_agg_per_user["last_auto_renew_transaction"]=pd.to_datetime(df_tra_agg_per_user["last_auto_renew_transaction"], format='%Y-%m-%d', errors='ignore')
print(df_tra_agg_per_user.dtypes)
df_tra_agg_per_user


msno                                                      object
total_num_transaction                                      int64
max_date_trans                                    datetime64[ns]
min_date_trans                                    datetime64[ns]
max_memb_exp                                      datetime64[ns]
min_amount_paid                                            int64
max_amount_paid                                            int64
mean_amount_paid                                         float64
median_amount_paid                                       float64
mode_amount_paid                                           int64
total_amount_paid                                          int64
total_plan_list_price                                      int64
mean_plan_list_price                                     float64
median_plan_list_price                                   float64
mode_plan_list_price                                       int64
num_payment_methods      

,msno,total_num_transaction,max_date_trans,min_date_trans,max_memb_exp,min_amount_paid,max_amount_paid,mean_amount_paid,median_amount_paid,mode_amount_paid,...,median_delta_trans_date,mode_delta_trans_date,min_delta_amount_paid,max_delta_amount_paid,mode_delta_amount_paid,duration_max_memb_exp_min_trans_date_effective,first_cancel_membership_expire_date,last_cancel_membership_expire_date,first_auto_renew_transaction,last_auto_renew_transaction
0,+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,1,2016-09-09,2016-09-09,2016-09-14,0,0,0.000000,0.0,0,...,NaN,NaN,NaN,NaN,NaN,5.0,NaT,NaT,NaT,NaT
1,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,2,2016-10-23,2015-11-21,2018-02-06,1599,1788,1693.500000,1693.5,1599,...,337.0,337.0,-189.0,-189.0,-189.0,808.0,NaT,NaT,NaT,NaT
2,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,5,2017-03-15,2016-11-16,2017-04-15,99,99,99.000000,99.0,99,...,30.0,31.0,0.0,0.0,0.0,150.0,NaT,NaT,2016-11-16,2017-03-15
3,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,21,2017-03-31,2015-01-31,2017-05-19,149,149,149.000000,149.0,149,...,31.0,31.0,0.0,0.0,0.0,695.0,NaT,NaT,2015-01-31,2017-03-31
4,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,27,2017-03-26,2015-01-26,2017-04-26,149,149,149.000000,149.0,149,...,31.0,31.0,0.0,0.0,0.0,821.0,NaT,NaT,2015-01-26,2017-03-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426138,zzz9+ZF4+GMyt63oU8xfjo1EkvRqH5OINlES0RUJI6I=,14,2016-11-03,2015-10-14,2016-11-13,129,149,131.857143,129.0,129,...,30.0,31.0,0.0,20.0,0.0,396.0,2016-11-03,2016-11-03,2015-10-14,2016-11-03
2426139,zzzF1KsGfHH3qI6qiSNSXC35UXmVKMVFdxkp7xmDMc0=,2,2017-03-05,2017-02-05,2017-04-04,149,149,149.000000,149.0,149,...,28.0,28.0,0.0,0.0,0.0,58.0,NaT,NaT,2017-02-05,2017-03-05
2426140,zzzN9thH22os1dRS0VHReY/8FTfGHOi86//d+wGGFsQ=,12,2017-02-02,2016-03-04,2017-02-04,99,99,99.000000,99.0,99,...,31.0,31.0,0.0,0.0,0.0,337.0,2017-02-02,2017-02-02,2016-03-04,2017-02-02
2426141,zzztsqkufVj9DPVJDM3FxDkhlbCL5z4aiYxgPSGkIK4=,1,2015-06-08,2015-06-08,2015-06-15,0,0,0.000000,0.0,0,...,NaN,NaN,NaN,NaN,NaN,7.0,NaT,NaT,NaT,NaT


In [60]:
df_user_all_info_agg=df_mem.merge(df_user_log_agg_read,on="msno")
print("after membership merge",len(df_user_all_info_agg))
df_user_all_info_agg=df_user_all_info_agg.merge(df_tra_agg_per_user,on="msno")
print("after transaction merge",len(df_user_all_info_agg))
df_user_all_info_agg

after membership merge 5169234
after transaction merge 1937764


,msno,city,bd,gender,registered_via,registration_init_time,registration_init_time_as_dt,activity_duration,last_date_active,n_active_days,...,median_delta_trans_date,mode_delta_trans_date,min_delta_amount_paid,max_delta_amount_paid,mode_delta_amount_paid,duration_max_memb_exp_min_trans_date_effective,first_cancel_membership_expire_date,last_cancel_membership_expire_date,first_auto_renew_transaction,last_auto_renew_transaction
0,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914,2011-09-14,820,2017-03-31,763,...,31.0,31.0,-20.0,20.0,0.0,846.0,2016-11-28,2016-11-28,2015-01-03,2017-03-27
1,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915,2011-09-15,14,2017-03-28,9,...,NaN,NaN,NaN,NaN,NaN,30.0,NaT,NaT,2017-03-14,2017-03-14
2,yLkV2gbZ4GLFwqTOXLVHz0VGrMYcgBGgKZ3kj9RiYu8=,4,30,male,9,20110916,2011-09-16,781,2017-03-31,18,...,31.0,31.0,0.0,0.0,0.0,672.0,NaT,NaT,2015-02-17,2017-03-31
3,WH5Jq4mgtfUFXh2yz+HrcTXKS4Oess4k4W3qKolAeb0=,5,34,male,9,20110916,2011-09-16,220,2017-03-30,73,...,179.0,179.0,0.0,0.0,0.0,366.0,NaT,NaT,NaT,NaT
4,I0yFvqMoNkM8ZNHb617e1RBzIS/YRKemHO7Wj13EtA0=,13,63,male,9,20110918,2011-09-18,817,2017-03-30,354,...,31.0,31.0,0.0,0.0,0.0,820.0,NaT,NaT,2015-01-27,2017-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1937759,OL6joplCKThE1pZD+AQDnj3kZiC45qzpndlDIRSyPQI=,1,0,NaN,7,20151020,2015-10-20,528,2017-03-31,396,...,31.0,31.0,0.0,0.0,0.0,548.0,NaT,NaT,2015-10-20,2017-03-20
1937760,O11rAa17ANM6DGMMVb9L2vwCQFv/S6SR3SDB1nGuOAc=,1,0,NaN,7,20151020,2015-10-20,525,2017-03-31,273,...,31.0,31.0,0.0,0.0,0.0,548.0,NaT,NaT,2015-10-20,2017-03-20
1937761,tyoRqQ72Y2oCcqvdaIxG8T0Hs9DZvuuM8IeTwhxtdBQ=,6,26,female,7,20151020,2015-10-20,528,2017-03-31,417,...,30.5,31.0,-99.0,149.0,0.0,536.0,2015-12-09,2015-12-09,2015-10-20,2017-03-08
1937762,VSGkb3hyBRUtb/b1MQUZbvOkktS3vKLnhMHW0CF8eyU=,1,0,NaN,7,20151020,2015-10-20,481,2017-02-15,2,...,26.0,26.0,0.0,0.0,0.0,31.0,2015-11-15,2015-11-15,2015-10-20,2015-11-15


In [65]:
for col in df_user_all_info_agg.columns:
    print(col,df_user_all_info_agg[col].isna().sum())

msno 0
city 0
bd 0
gender 958307
registered_via 0
registration_init_time 0
registration_init_time_as_dt 0
activity_duration 0
last_date_active 0
n_active_days 0
sum_num_25 0
sum_num_50 0
sum_num_75 0
sum_num_985 0
sum_num_100 0
sum_num_unq 0
tot_secs 0
mean_num_25 0
median_num_25 0
q25_num_25 0
q75_num_25 0
std_num_25 87507
median_num_50 0
std_num_50 87507
median_num_75 0
std_num_75 87507
median_num_985 0
std_num_985 87507
mean_num_100 0
median_num_100 0
q25_num_100 0
q75_num_100 0
std_num_100 87507
mean_tot_secs 0
median_tot_secs 0
q25_tot_secs 0
q75_tot_secs 0
std_tot_secs 87507
n_max_break 87507
n_min_break 87507
n_mean_break 87507
n_mode_break 87507
n_median_break 87507
n_max_break_last 87507
n_min_break_last 87507
n_mean_break_last 87507
n_mode_break_last 87507
n_median_break_last 87507
total_num_transaction 0
max_date_trans 0
min_date_trans 0
max_memb_exp 0
min_amount_paid 0
max_amount_paid 0
mean_amount_paid 0
median_amount_paid 0
mode_amount_paid 0
total_amount_paid 0
total_pla

In [70]:
df_user_all_info_agg[df_user_all_info_agg["std_tot_secs"].isna()]["mean_tot_secs"].max()

1271346.5259999998

In [63]:
df_user_all_info_agg_outer=df_mem.merge(df_user_log_agg_read,on="msno",how="outer")
print("after membership merge_outer",len(df_user_all_info_agg_outer))
df_user_all_info_ag_outerg=df_user_all_info_agg_outer.merge(df_tra_agg_per_user,on="msno",how="outer")
print("after transaction merge_outer",len(df_user_all_info_ag_outerg))
df_user_all_info_agg_outer

after membership merge_outer 6939661
after transaction merge_outer 7377004


,msno,city,bd,gender,registered_via,registration_init_time,registration_init_time_as_dt,activity_duration,last_date_active,n_active_days,...,n_max_break,n_min_break,n_mean_break,n_mode_break,n_median_break,n_max_break_last,n_min_break_last,n_mean_break_last,n_mode_break_last,n_median_break_last
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1.0,0.0,NaN,11.0,20110911.0,2011-09-11,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1.0,0.0,NaN,7.0,20110914.0,2011-09-14,820.0,2017-03-31,763.0,...,7.0,0.0,0.076115,0.0,0.0,1.0,0.0,0.034483,0.0,0.0
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1.0,0.0,NaN,11.0,20110915.0,2011-09-15,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1.0,0.0,NaN,11.0,20110915.0,2011-09-15,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6.0,32.0,female,9.0,20110915.0,2011-09-15,14.0,2017-03-28,9.0,...,3.0,0.0,0.750000,0.0,0.0,3.0,0.0,0.750000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6939656,PQBDtyEUkJdv7Zb0ebClh0WhH/WIu11XQNU5sEcetP8=,NaN,NaN,NaN,NaN,NaN,NaT,0.0,2017-03-03,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6939657,QQapErUBXq59Q+8chH1yTEzEpfT4YfRx9R/P2ToYXaA=,NaN,NaN,NaN,NaN,NaN,NaT,0.0,2017-03-19,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6939658,QuCXSKyV44OS+L4ofixj9QUgSrwB7DSQRd/YWhHL1hg=,NaN,NaN,NaN,NaN,NaN,NaT,1.0,2017-03-31,2.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
6939659,gmhNhbDfUgBur7/pwg8xLdlTOFzRVwtsJZsi6tdrAxE=,NaN,NaN,NaN,NaN,NaN,NaT,0.0,2017-03-05,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df_train=pd.read_csv("data/train.csv")
df_train.sort_values(by=["msno"])
df_train.nunique()

msno        992931
is_churn         2
dtype: int64

In [73]:
df_train_v2=pd.read_csv("data/train_v2.csv")
df_train_v2.sort_values(by=["msno"])
df_train_v2.nunique()

msno        970960
is_churn         2
dtype: int64

Out of 970 960 users in train dataset, 855 160 are available in the user log dataset, 860 967 are available in the membership dataset, 855 136 are available in each of the user log, transaction, and membership data. All of training data is available in the transaction dataset.

In [79]:
df_test=df_train_v2.merge(df_tra_agg_per_user, on='msno')
df_test

,msno,is_churn,total_num_transaction,max_date_trans,min_date_trans,max_memb_exp,min_amount_paid,max_amount_paid,mean_amount_paid,median_amount_paid,...,median_delta_trans_date,mode_delta_trans_date,min_delta_amount_paid,max_delta_amount_paid,mode_delta_amount_paid,duration_max_memb_exp_min_trans_date_effective,first_cancel_membership_expire_date,last_cancel_membership_expire_date,first_auto_renew_transaction,last_auto_renew_transaction
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,6,2017-02-28,2015-03-31,2017-03-30,149,149,149.000000,149.0,...,31.0,31.0,0.0,0.0,0.0,156.0,2015-09-01,2015-09-01,2015-03-31,2015-09-02
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,6,2017-03-11,2016-03-03,2017-04-11,0,180,134.500000,149.0,...,34.0,2.0,-149.0,180.0,0.0,160.0,NaT,NaT,NaT,NaT
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,20,2017-03-14,2015-08-19,2017-06-15,0,300,149.100000,149.0,...,32.0,33.0,-149.0,300.0,0.0,647.0,NaT,NaT,NaT,NaT
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,28,2016-02-25,2015-01-08,2018-01-08,0,149,143.678571,149.0,...,17.0,0.0,-149.0,0.0,0.0,1082.0,2014-03-08,2016-02-25,2015-01-08,2016-02-25
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,32,2017-03-16,2015-01-24,2017-09-18,99,149,117.125000,114.0,...,30.0,31.0,-30.0,30.0,0.0,968.0,2015-09-29,2017-06-20,2015-01-24,2017-03-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970955,OHnZbu+EVaP+vN7Z+OfT5OMcp90MWFZonmM0o3pb8FY=,0,27,2017-03-04,2015-01-03,2017-04-03,0,149,143.481481,149.0,...,31.0,31.0,0.0,149.0,0.0,821.0,NaT,NaT,2015-01-03,2017-03-04
970956,S92bDK//uI6hk3u1vuApro0qJBQOToBozZ7lL1yTC+w=,0,9,2017-03-10,2016-07-10,2017-04-10,99,99,99.000000,99.0,...,31.0,31.0,0.0,0.0,0.0,274.0,NaT,NaT,2016-07-10,2017-03-10
970957,eUa3xo16vpAjr43Cjlb6Kjf1NTILYyJIkBayJQdXWnw=,0,20,2017-03-16,2015-08-16,2017-04-16,99,99,99.000000,99.0,...,31.0,31.0,0.0,0.0,0.0,609.0,NaT,NaT,2015-08-16,2017-03-16
970958,iZE41tbAQ65rJq60olkJT4BJzuUAYgQdfbEemXe/TTk=,0,21,2017-03-31,2015-02-14,2017-05-13,149,149,149.000000,149.0,...,31.0,31.0,0.0,0.0,0.0,663.0,NaT,NaT,2015-02-14,2017-03-31


In [74]:
df_user_all_info_agg_for_model=df_user_all_info_agg.merge(df_train_v2,on="msno")
df_user_all_info_agg_for_model

,msno,city,bd,gender,registered_via,registration_init_time,registration_init_time_as_dt,activity_duration,last_date_active,n_active_days,...,mode_delta_trans_date,min_delta_amount_paid,max_delta_amount_paid,mode_delta_amount_paid,duration_max_memb_exp_min_trans_date_effective,first_cancel_membership_expire_date,last_cancel_membership_expire_date,first_auto_renew_transaction,last_auto_renew_transaction,is_churn
0,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914,2011-09-14,820,2017-03-31,763,...,31.0,-20.0,20.0,0.0,846.0,2016-11-28,2016-11-28,2015-01-03,2017-03-27,0
1,yLkV2gbZ4GLFwqTOXLVHz0VGrMYcgBGgKZ3kj9RiYu8=,4,30,male,9,20110916,2011-09-16,781,2017-03-31,18,...,31.0,0.0,0.0,0.0,672.0,NaT,NaT,2015-02-17,2017-03-31,0
2,I0yFvqMoNkM8ZNHb617e1RBzIS/YRKemHO7Wj13EtA0=,13,63,male,9,20110918,2011-09-18,817,2017-03-30,354,...,31.0,0.0,0.0,0.0,820.0,NaT,NaT,2015-01-27,2017-03-27,0
3,OoDwiKZM+ZGr9P3fRivavgOtglTEaNfWJO4KaJcTTts=,1,0,NaN,7,20110918,2011-09-18,817,2017-03-28,295,...,31.0,0.0,0.0,0.0,822.0,NaT,NaT,2015-01-26,2017-03-26,1
4,4De1jAxNRABoyRBDZ82U0yEmzYkqeOugRGVNIf92Xb8=,4,28,female,9,20110920,2011-09-20,761,2017-03-31,37,...,31.0,0.0,31.0,0.0,274.0,2015-06-07,2015-06-07,2015-03-31,2017-03-03,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855131,dz8iQV2eZCBZo1DjKL7OifsiyzGf3xyMaCFDOJHVeyc=,1,0,NaN,7,20151020,2015-10-20,528,2017-03-31,108,...,31.0,0.0,0.0,0.0,547.0,NaT,NaT,2015-10-20,2017-03-19,0
855132,OL6joplCKThE1pZD+AQDnj3kZiC45qzpndlDIRSyPQI=,1,0,NaN,7,20151020,2015-10-20,528,2017-03-31,396,...,31.0,0.0,0.0,0.0,548.0,NaT,NaT,2015-10-20,2017-03-20,0
855133,O11rAa17ANM6DGMMVb9L2vwCQFv/S6SR3SDB1nGuOAc=,1,0,NaN,7,20151020,2015-10-20,525,2017-03-31,273,...,31.0,0.0,0.0,0.0,548.0,NaT,NaT,2015-10-20,2017-03-20,0
855134,tyoRqQ72Y2oCcqvdaIxG8T0Hs9DZvuuM8IeTwhxtdBQ=,6,26,female,7,20151020,2015-10-20,528,2017-03-31,417,...,31.0,-99.0,149.0,0.0,536.0,2015-12-09,2015-12-09,2015-10-20,2017-03-08,0


In [80]:
df_user_all_info_agg_for_model.to_csv("df_user_log__transfer__membership__aggregated_per_user__training_members.csv")